# Eflux2 applied to *S. elongatus*

In [33]:
import sys
sys.path.append('../src')
sys.path.append('/Users/mahs128/Repos/CONCERTO')
from concerto.helpers.load_model_from_git import load_model_from_git
from eflux2 import EFlux2
import cobra
import pandas as pd
import numpy as np
import gurobipy

## Load in inputs

### Read in transcriptomics data

In [30]:
transcriptomics_fname = "processed_data/cleaned_transcriptomics.csv"
transcriptomics_df = pd.read_csv(transcriptomics_fname, index_col="Label")
transcriptomics_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Label,,,,,,,,,
SYNPCC7942_RS00005,16290,17761,15101,14567,13967,12653,7016,8855,8689
SYNPCC7942_RS00010,7185,7502,6565,10086,7546,7705,3747,6670,6960
SYNPCC7942_RS00015,24176,26304,22781,23926,21306,20412,13440,17253,17053
SYNPCC7942_RS00020,35119,35145,25895,37701,34175,30569,24835,39769,30280
SYNPCC7942_RS00025,6891,7803,6607,6347,6284,6844,3326,4146,3180


### Load SBML model

In [31]:
# syn_model = load_model_from_git('Synechococcus')
syn_model = cobra.io.read_sbml_model('../models/iJB785_w_sucrose_transport.xml')

In [32]:
type(syn_model)

cobra.core.model.Model

### Load rates

In [5]:
### TODO: MISSING RATES FOR SYN MODEL ###
rates_fname = "processed_data/cleaned_metabolomic_abundance_rates.csv"
rates_df = pd.read_csv(rates_fname, index_col="Sample")
rates_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Sample,,,,,,,,,
13ppd,-29836.5,-61719.5,12502.0,20379.0,31300.5,-10967.5,3339.0,-34320.5,12960.0
3hbz,-2978.0,-1145.0,-113.5,571.5,219.0,444.5,-437.0,-1084.5,-930.0
4hbz,-2529.5,1568.5,-1020.5,1889.5,-1916.0,810.0,-2246.5,818.0,1859.5
aso4,-4807.5,-3176.0,275.0,-795.0,-1199.0,-3490.0,1443.0,-2645.5,4379.5
bz,-83691.0,-34668.5,-18102.5,-13110.5,-25685.0,-52963.5,15482.5,-2626.0,21771.0


## Normalize relevant data with respect to reference strain/line

### Identify reference strain/line

In [11]:
ref_rep = transcriptomics_df.columns[4] # 'Se_axen_d6_2'
ref_rep

'Se_axen_d6_2'

### Normalize transcriptomics data

In [15]:
normalized_transcriptomics_df = transcriptomics_df.div(transcriptomics_df[ref_rep], axis=0)
normalized_transcriptomics_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Label,,,,,,,,,
SYNPCC7942_RS00005,1.166321,1.271640,1.081191,1.042958,1.0,0.905921,0.502327,0.633994,0.622109
SYNPCC7942_RS00010,0.952160,0.994169,0.869997,1.336602,1.0,1.021071,0.496554,0.883912,0.922343
SYNPCC7942_RS00015,1.134704,1.234582,1.069229,1.122970,1.0,0.958040,0.630808,0.809772,0.800385
SYNPCC7942_RS00020,1.027623,1.028383,0.757718,1.103175,1.0,0.894484,0.726701,1.163687,0.886028
SYNPCC7942_RS00025,1.096595,1.241725,1.051400,1.010025,1.0,1.089115,0.529281,0.659771,0.506047


### Normalize rates data

In [16]:
normalized_rates_df = rates_df.div(rates_df[ref_rep], axis=0)
normalized_rates_df.head()

,Se_axen_d4_1,Se_axen_d4_2,Se_axen_d4_3,Se_axen_d6_1,Se_axen_d6_2,Se_axen_d6_3,Se_axen_d8_1,Se_axen_d8_2,Se_axen_d8_3
Sample,,,,,,,,,
13ppd,-0.953228,-1.971838,0.399419,0.651076,1.0,-0.350394,0.106676,-1.096484,0.414051
3hbz,-13.598174,-5.228311,-0.518265,2.609589,1.0,2.029680,-1.995434,-4.952055,-4.246575
4hbz,1.320198,-0.818633,0.532620,-0.986169,1.0,-0.422756,1.172495,-0.426931,-0.970511
aso4,4.009591,2.648874,-0.229358,0.663053,1.0,2.910759,-1.203503,2.206422,-3.652627
bz,3.258361,1.349757,0.704789,0.510434,1.0,2.062040,-0.602784,0.102239,-0.847615


## Calculate fluxes

### Explore candidate uptake reactions

#### Exploring Carbon/Biomass
This would be a typical approach, but measurements for neither CO2 nor HCO3 are available in the metabolics data. Therefore, we can either (a) ignore uptake reactions and not use them to narrow the bounds for flux calculations, or (b) use growth rates instead


In [17]:
# # Explore CO2 exchange and transport from external compartment --> periplasm --> cytoplasm

# print("----")
# print("co2_e reactions")
# print("----")
# for r in syn_model.metabolites.co2_e.reactions:
#     print(r.id)

# print(" ")
# print("----")
# print("co2_p reactions")
# print("----")
# for r in syn_model.metabolites.co2_p.reactions:
#     print(r.id)



In [18]:
# display(syn_model.reactions.get_by_id('EX_co2_e'))
# display(syn_model.reactions.get_by_id('CO2tex'))
# display(syn_model.reactions.get_by_id('CO2tpp')) 

### Explore candiate secretion reactions

#### Exploring Sucrose output

In [19]:
# Explore sucrose exchange and transport from cytoplasm --> external compartment
print("----")
print("sucr_c reactions")
print("----")
for r in syn_model.metabolites.sucr_c.reactions:
    print(r.id)

print(" ")
print("----")
print("sucr_e reactions")
print("----")
for r in syn_model.metabolites.sucr_e.reactions:
    print(r.id)

----
sucr_c reactions
----
SUCR
SUCRt2
SPP
 
----
sucr_e reactions
----
EX_sucr_e
SUCRt2


In [20]:
display(syn_model.reactions.get_by_id('SUCR'))
display(syn_model.reactions.get_by_id('SUCRt2'))
display(syn_model.reactions.get_by_id('SPP'))
display(syn_model.reactions.get_by_id('EX_sucr_e')) # candidate for secretion target

Reaction identifier,SUCR
Name,Sucrose hydrolyzing enzyme
Memory address,0x2a598fe10
Stoichiometry,h2o_c + sucr_c --> fru_c + glc__D_c H2O H2O + Sucrose C12H22O11 --> D-Fructose + D-Glucose
GPR,SYNPCC7942_RS02025
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,SUCRt2
Name,Sucrose transport in via proton symport
Memory address,0x2a59e2350
Stoichiometry,h_e + sucr_e <=> h_c + sucr_c H+ + Sucrose C12H22O11 <=> H+ + Sucrose C12H22O11
GPR,ATCC_700927
Lower bound,-1000.0
Upper bound,1000.0


Reaction identifier,SPP
Name,Sucrose-phosphate phosphatase
Memory address,0x2a56f3d50
Stoichiometry,h2o_c + suc6p_c --> pi_c + sucr_c H2O H2O + Sucrose 6-phosphate --> Phosphate + Sucrose C12H22O11
GPR,SYNPCC7942_RS04180
Lower bound,0.0
Upper bound,1000.0


Reaction identifier,EX_sucr_e
Name,Sucrose exchange
Memory address,0x2a59d7e10
Stoichiometry,sucr_e --> Sucrose C12H22O11 -->
GPR,
Lower bound,0.011
Upper bound,1000.0


In [21]:
# Identify candiates for target intake metabolite and corresponding reaction
intake_target_met = None #
# intake_target_rxn = ""

# Identify candiates for target secretion metabolite and corresponding reaction
secrete_target_met = "sucr" # BiGG ID for sucrose
secrete_target_rxn = 'EX_sucr_e'

# List of replicates/strains/lines included in data
replicate_list = normalized_transcriptomics_df.columns

# Use teh Gurobi solver in the model
syn_model.solver = 'gurobi'

# TODO: generalize this as a function with various options for intake candidate availability, or using growth rates instead

fluxes = {}
for rep in replicate_list:
    with syn_model:
        # Collect rates for this replicate/strain/line

        # TODO: condition intake/secrete targets if a candidate is chosen
        # intake_target = syn_model.reactions.id....[co2_intake] 
        secrete_target = normalized_rates_df.loc[secrete_target_met, rep]
      #  syn_model.reactions.get_by_id(secrete_target_rxn) # secrete_3hp = normalized_line_rates.loc[rep,'3hp_secretion_rates (mmol/gDCW * hr)']
       
        # Update model reaction bounds with experimentally calculated rate
        # if intake_target:
        #     getattr(syn_model.reactions, intake_target_rxn).reaction.upper_bound = intake_target
        if secrete_target:
            getattr(syn_model.reactions, secrete_target_rxn).lower_bound = secrete_target
        try:
            print(rep)
            fluxes[rep] = EFlux2(syn_model, normalized_transcriptomics_df[rep])
        except TypeError:
            print(f"Replicate {rep} with uptake of {intake_target_met} and secretion of {secrete_target_met} is infeasible")

Se_axen_d4_1
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpclgy9w5t.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros
FBA status optimal
FBA solution 0.01375879927219083
EFlux2 status optimal
EFlux2 solution 11.583086730834395
Se_axen_d4_2
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmps7hzhc1k.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros
FBA status infeasible
FBA solution None
Replicate Se_axen_d4_2 with uptake of None and secretion of sucr is infeasible
Se_axen_d4_3
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpipax7prv.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros


/Users/mahs128/.pyenv/versions/3.11.5/envs/bmca/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FBA status infeasible
FBA solution None
Replicate Se_axen_d4_3 with uptake of None and secretion of sucr is infeasible
Se_axen_d6_1
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpbdgy2d71.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros
FBA status infeasible
FBA solution None
Replicate Se_axen_d6_1 with uptake of None and secretion of sucr is infeasible
Se_axen_d6_2
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpmlk2e8vs.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros
FBA status infeasible
FBA solution None
Replicate Se_axen_d6_2 with uptake of None and secretion of sucr is infeasible
Se_axen_d6_3
Read LP format model from file /var/folders/k9/b8pxky2572sdtgy2vnxhcljw0000gn/T/tmpzf1kdev0.lp
Reading time = 0.01 seconds
: 769 rows, 1702 columns, 7652 nonzeros
FBA status optimal
FBA solution 0.043907541747089524
EFlux2 status optimal
EFlux2 solution 60.56797457695283
Se_axe

KeyError: 'Se_axen_d8_3'

In [22]:
fluxes

{'Se_axen_d4_1': <Solution 11.583 at 0x2a757b5d0>,
 'Se_axen_d6_3': <Solution 60.568 at 0x2a7587cd0>,
 'Se_axen_d8_2': <Solution 41.327 at 0x2a9232890>}

In [23]:
fluxes['Se_axen_d8_2']

,fluxes,reduced_costs
EX_gln__L_e,0.000000,2801.156582
EX_hco3_e,-0.233722,0.000000
EX_mn2_e,0.000000,0.000000
EX_arg__L_e,0.000000,6844.619332
ADPT,0.000007,0.000000
...,...,...
MDDEP3pp,0.000000,0.000000
MDDEP4pp,0.000000,0.000000
MNHNAtpp,0.099447,0.000000
EX_sucr_e,-0.378780,0.000000


In [28]:
# Save fluxes to file for the following format:
# row names = metabolites
# column names = replicates
# Entries "fluxes" column values from computed fluxes variable above
flux_df = pd.DataFrame({
    rep: fluxes[rep].fluxes for rep in fluxes
})
flux_df.to_csv("processed_data/Eflux_for_Syn.csv")